# <img src="https://full-stack-assets.s3.eu-west-3.amazonaws.com/M03-EDA/Tinder-Symbole.png" alt="Alt text" style="width:100px;"/>

**<font color="white">Tinder</font>**, une application de rencontres en ligne et de réseautage géosocial, a été lancée en 2012 par **<font color="white">Sean Rad</font>** lors d'un hackathon à West Hollywood. Les utilisateurs de Tinder peuvent "swiper à droite" pour **<font color="white">aimer</font>** ou "swiper à gauche" pour **<font color="white">rejeter</font>** les profils d'autres utilisateurs, facilitant ainsi les **<font color="white">rencontres en ligne</font>**.

Mettons nous dans la peau du **<font color="white">département marketing de Tinder</font>**, qui cherche à comprendre ce qui suscite l'intérêt des utilisateurs les uns pour les autres, car le nombre de correspondances diminue. Ils ont mené une expérience de speed dating pour collecter des données sur les rencontres entre participants et leur **<font color="white">volonté d'accepter un deuxième rendez-vous</font>**.

Les données proviennent d'événements de speed dating expérimentaux entre 2002 et 2004. Chaque ligne du jeu de données représente un speed date entre deux personnes et indique si elles ont accepté d'aller à un deuxième rendez-vous ensemble. Les données comprennent également des informations démographiques, des habitudes de rencontres et des perceptions personnelles.


## Install and import lib

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from scipy import stats
import plotly.express as px
from geopy.geocoders import Photon
import pandas as pd
import plotly.graph_objects as go

## Data loading
Chargement du fichier de données issu d'un csv

In [3]:
df = pd.read_csv("Speed_Dating_Data.csv",sep=",",encoding='latin-1')

## Data Observing

En premier lieu, nous devons d'observer afin de comprendre le dataset. Regardons quelques colonnes importantes, les match, le genre, les traits de personnalités (attributes), l'objectif à travers ce dates. Ceci permettra de nous orienter vers notre objectif de comprendre pourquoi un prétendant souhaite passer à un deuxième date.

In [4]:
df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [5]:
df[df['gender'] == 0][['attr1_1','sinc1_1','intel1_1','fun1_1','amb1_1','shar1_1']].head()

,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1
0,15.0,20.0,20.0,15.0,15.0,15.0
1,15.0,20.0,20.0,15.0,15.0,15.0
2,15.0,20.0,20.0,15.0,15.0,15.0
3,15.0,20.0,20.0,15.0,15.0,15.0
4,15.0,20.0,20.0,15.0,15.0,15.0


## Choose config


1_ | Attribut recherché chez l'autre sexe

2_ | Attribut recherché par l'autre sexe

3_ | Auto-évaluation des attributs (basée sur votre propre opinion)

4_ | Attribut recherché par d'autres personnes de votre sexe

5_ | Auto-évaluation des attributs (en fonction de ce qu'ils pensaient que les autres percevaient)

7_ | Notation de l'importance des attributs en termes de contribution à vos décisions

///////

_1 | Rempli avant l'événement

_2 | Rempli un jour après l'événement

_3 | Rempli 3-4 semaines après l'événement

_s | Rempli à mi-parcours de l'événement

In [6]:
#attributA_B
a,b = 1,2


## Selecting/Filtering Data

L'objectif est d'enelver les colonnes qui nous ne serviront pas. En utilisant la fonction dropna() ou en utilisant des filtres.

Supprimer les colonnes ou l'on a plus de 50% de NaN type, passant de 195 à 136. On fait le tri des colonnes, lorsque les colonnes sont trop manquantes je les enlève du dataset.

In [7]:
threshold = int(len(df)/2)
df.dropna(axis=1, thresh=threshold, inplace=True)
df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2
0,1,1.0,0,1,1,1,10,7,NaN,4,...,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN


In [8]:
city_df = df[['from','id']].groupby('from').count().reset_index()['from']
geolocator = Photon(user_agent="measurements")

def do_geocode(address):
    try:
        location = geolocator.geocode(address)
        return location.latitude, location.longitude
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None

coordinates = city_df.apply(do_geocode)

coordinates_df = pd.DataFrame(coordinates.tolist(), columns=['Latitude', 'Longitude'])
coordinates_df['z'] = df[['from', 'id']].groupby('from').count().reset_index()['id']
coordinates_df = coordinates_df.dropna()

max_z_index = coordinates_df['z'].idxmax()

center_lat = coordinates_df.loc[max_z_index, 'Latitude']
center_lon = coordinates_df.loc[max_z_index, 'Longitude']


Error geocoding Born in Montana, raised in South Jersey (nr. Philadelphia): 'NoneType' object has no attribute 'latitude'
Error geocoding Katonah, NY (more recently, Boston): 'NoneType' object has no attribute 'latitude'
Error geocoding Midwest USA: 'NoneType' object has no attribute 'latitude'
Error geocoding NYC-6 yrs. Grew up in Nebraska: 'NoneType' object has no attribute 'latitude'
Error geocoding way too little space here. world citizen.: 'NoneType' object has no attribute 'latitude'


Nombre de personnes classées par objectifs en allant au date :

- Ça avait l'air d'être une soirée amusante = 1
- Pour rencontrer de nouvelles personnes = 2
- Pour obtenir un date = 3
- En recherche d'une relation sérieuse=4
- Dire que je l'ai fait =5
- Autre=6

In [9]:
df.groupby(['goal']).count().reset_index()['id']

0    3425
1    3012
2     631
3     301
4     510
5     419
Name: id, dtype: int64

Selection des traits de personnalités

In [10]:
df.filter(regex='^(attr|sinc|intel|fun|amb|shar)').head()

,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,attr1_1,sinc1_1,intel1_1,fun1_1,...,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2
0,6.0,8.0,8.0,8.0,8.0,6.0,15.0,20.0,20.0,15.0,...,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN
1,7.0,8.0,10.0,7.0,7.0,5.0,15.0,20.0,20.0,15.0,...,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN
2,10.0,10.0,10.0,10.0,10.0,10.0,15.0,20.0,20.0,15.0,...,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN
3,7.0,8.0,9.0,8.0,9.0,8.0,15.0,20.0,20.0,15.0,...,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN
4,8.0,7.0,9.0,6.0,9.0,7.0,15.0,20.0,20.0,15.0,...,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN


Selection des attributs en fonction de la configuration souhaitée et statistiques descriptives.

In [30]:
attributes_col = [f'attr{a}_{b}',f'sinc{a}_{b}',f'intel{a}_{b}',f'fun{a}_{b}',f'amb{a}_{b}',f'shar{a}_{b}']
df_attributes = df[attributes_col]
df_attributes.describe()

,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2
count,7445.000000,7463.000000,7463.000000,7463.000000,7463.000000,7463.000000
mean,26.217194,15.865084,17.813755,17.654765,9.913436,12.760263
std,14.388694,6.658494,6.535894,6.129746,5.675550,6.651547
min,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16.670000,10.000000,15.000000,15.000000,5.000000,10.000000
50%,20.000000,16.670000,19.050000,18.370000,10.000000,13.000000
75%,30.000000,20.000000,20.000000,20.000000,15.000000,16.670000
max,85.000000,50.000000,40.000000,50.000000,22.220000,35.000000


**Quels sont les attributs les moins désirables chez un partenaire masculin ? Est-ce différent pour les partenaires féminines ?**

En regardant la moyenne, cela semble être l'ambition. Mais la médiane est plus pertinente que la moyenne, ici la médiane montre que les intérêts et les passe-temps partagés ne sont pas les attributs les plus importants/souhaitables recherchés par l'homme.

In [32]:
attributes_col_id = ['id',f'attr{a}_{b}',f'sinc{a}_{b}',f'intel{a}_{b}',f'fun{a}_{b}',f'amb{a}_{b}',f'shar{a}_{b}']
df_attributes_id = df[attributes_col_id]
df_attributes_id.groupby('id').mean().reset_index().head()


,id,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2
0,1.0,22.328596,18.058713,18.798070,17.974932,10.437427,12.421969
1,2.0,26.285931,15.057085,17.133806,17.545668,11.107652,12.993077
2,3.0,24.226839,17.493118,17.465978,17.787269,10.423871,12.602925
3,4.0,26.195051,16.553901,17.452177,18.068973,9.725133,12.970513
4,5.0,28.069422,14.269850,18.451863,17.078994,9.101049,13.285375


In [33]:
#Compare man and women attributes looking to other partner
attributes_col_gender = ['gender','id',f'attr{a}_{b}',f'sinc{a}_{b}',f'intel{a}_{b}',f'fun{a}_{b}',f'amb{a}_{b}',f'shar{a}_{b}']
df_attributes_gender = df[attributes_col_gender]
df_attributes_gender.groupby(['gender']).mean().reset_index().groupby(['gender']).describe()
df_attributes_gender.groupby(['id','gender']).mean().reset_index().groupby(['gender']).mean().reset_index()


,gender,id,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2
0,0,11.5,21.477421,16.919138,19.005129,17.387060,11.174608,14.702891
1,1,11.5,32.629313,14.018271,16.499483,17.247019,8.258303,11.319110


In [34]:
df_attributes_gender.groupby(['id','gender']).mean().reset_index().groupby(['gender']).mean().reset_index()

,gender,id,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2
0,0,11.5,21.477421,16.919138,19.005129,17.387060,11.174608,14.702891
1,1,11.5,32.629313,14.018271,16.499483,17.247019,8.258303,11.319110


In [36]:
df_attributes

,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2
0,19.44,16.67,13.89,22.22,11.11,16.67
1,19.44,16.67,13.89,22.22,11.11,16.67
2,19.44,16.67,13.89,22.22,11.11,16.67
3,19.44,16.67,13.89,22.22,11.11,16.67
4,19.44,16.67,13.89,22.22,11.11,16.67
...,...,...,...,...,...,...
8373,70.00,0.00,15.00,10.00,0.00,5.00
8374,70.00,0.00,15.00,10.00,0.00,5.00
8375,70.00,0.00,15.00,10.00,0.00,5.00
8376,70.00,0.00,15.00,10.00,0.00,5.00


## Descriptive Statistic

Avoir une première idée de données à disposition pour comprendre le comportement et par la suite faire des plots qui illustrent ces statistiques.

In [17]:
df[['attr1_1','sinc1_1','intel1_1','fun1_1','amb1_1','shar1_1']].describe()

,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1
count,8299.000000,8299.000000,8299.000000,8289.000000,8279.000000,8257.000000
mean,22.514632,17.396389,20.265613,17.457043,10.682539,11.845111
std,12.587674,7.046700,6.783003,6.085239,6.124888,6.362154
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,15.000000,15.000000,17.390000,15.000000,5.000000,9.520000
50%,20.000000,18.180000,20.000000,18.000000,10.000000,10.640000
75%,25.000000,20.000000,23.810000,20.000000,15.000000,16.000000
max,100.000000,60.000000,50.000000,50.000000,53.000000,30.000000


In [18]:
fig = px.histogram(df,x='age',color='gender',width=700,height=500)
fig.show()

In [19]:
fig = go.Figure(go.Densitymapbox(
    lat=coordinates_df['Latitude'],
    lon=coordinates_df['Longitude'],
    z=coordinates_df['z'],
    radius=10,
    colorscale='Viridis',
    colorbar=dict(
        title='Subjects'
    )
))

fig.update_layout(
    mapbox_style='carto-positron',
    mapbox_center={'lat': center_lat, 'lon': center_lon},
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    height=400,
    width=800
)

fig.show()

In [46]:
df_goal = df.groupby(['goal']).count().reset_index()
# Define a dictionary mapping numbers to strings
replace_dict = {
    1: "Seemed like a fun night out",
    2: "To meet new people",
    3: "To get a date",
    4: "Looking for a serious relationship",
    5: "To say I did it",
    6: "Other"
}

df_goal['goal'] = df_goal['goal'].replace(replace_dict)

color_palette = px.colors.sequential.Blues[::-1]

fig = px.pie(df_goal, names = df_goal['goal'],values = df_goal['id'],
             color_discrete_sequence=color_palette,width=700,height=500)
fig.show()

Répartiton des points attribués de 0 à 100 sur les 6 traits de personnalités.

In [21]:
df_attributes_agg = df_attributes.agg({'median', 'mean','std'}).reset_index()
df_attributes_agg

,index,id,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2
0,mean,9.024140,21.890754,16.696773,19.021396,17.463838,11.412737,14.037736
1,median,8.000000,20.000000,17.650000,20.000000,17.860000,10.000000,15.000000
2,std,5.574964,11.442558,6.330506,6.097872,5.987142,5.206001,6.683525


## Data visualisation

In [23]:

fig = px.pie(df_attributes, names=attributes_col, values=df_attributes.median(), 
             color_discrete_sequence=color_palette,width=700,height=500)

fig.show()


In [24]:

# Reshape the DataFrame to long format
df_long = pd.melt(df, id_vars=['gender'], value_vars=df_attributes.iloc[:,-6:].columns,
                   var_name='attribute', value_name='value')
# df_long

In [25]:
fig = px.box(df_long, x="attribute", y="value", color="gender",
             labels={"value": "Valeur de l'attribut", "attribute": "Attribut", "gender": "Genre"},
             title="Boxplots par attribut et par genre")

new_names = {'0': 'Femme', '1': 'Homme'}

# Mettre à jour les noms des étiquettes
fig.for_each_trace(lambda t: t.update(name=new_names[t.name], legendgroup=new_names[t.name], 
                                       hovertemplate=t.hovertemplate.replace(t.name, new_names[t.name]) if t.name in new_names else t.hovertemplate))

fig.update_layout(height=600, width=1000) 
fig.update_yaxes(range=[-5, 105])

fig.show()

**Goal:**
What is your primary goal in participating in this event?  

- Seemed like a fun night out = 1  
- To meet new people = 2  
- To get a date = 3  
- Looking for a serious relationship = 4  
- To say I did it = 5  
- Other = 6  


In [43]:
var1 ='match'
var2 ='goal'

df_sunburst = df[[var1,var2,'id']].groupby([var1,var2]).count().reset_index()
goal = df_sunburst[var2]
match = df_sunburst[var1]
match_count = df_sunburst['id']

dft = pd.DataFrame(
    dict(goal=goal, match=match, match_count=match_count)
)
goal_mapping = {
    1: 'Fun night out',
    2: 'Meet people',
    3: 'Get a date',
    4: 'Serious relationship',
    5: 'To say I did it',
    6: 'Other'
}

dft['goal'] = dft['goal'].map(goal_mapping)

fig = px.sunburst(dft, path=[var1,var2], values='match_count')

fig.update_layout(
    title=f"{var1} --> {var2}",
    title_font_size=24, 
    title_font_family="Gotham",
    width=800,
    height=600,    
)
fig.show()

## Conclusion

Les facteurs qui montrent une envie de passer au second date sont aussi nombreux que le nombre de personnes présents lors de cette récolte de données. Cependant, nous pouvons observer quelques tendances et quelques facteurs aportant des éléments de réponse à la problématique. Dans ce notebook nous sommes intéressés à une dizaine de paramètres sur les 195 présents initialement. Notamment des paramètres de traits de personnalités, genre 

En effet, nous avons suivi la démarche suivante:
- installation et importation des librairies
- chargement des données
- compréhension, nettoyage et filtrage des données
- sélection et préparation des données pour leur analyse
- description statistiques 
- visualisation des données 

Cette dernière dont le rôle primordial permet d'améliorer la compréhension de la description statistique du jeu de données.


Cette étude rescence essentiellement des jeunes américains de de 18 à 36 ans. Pour Les deux principaux objectifs en venant à cette soirée était de rencontrer de nouvelles personnes (36%) ou bien parce que cette soirée semblait amusante (41%).


**Conclusion des analyses !**
